### 1.实战练习
我们利用之前学习的内容来实战练习一个简单的Chat Bot，同时我们会实现一下简单的memory功能，即记录用户的历史对话。此处主要用到了 Reducer 函数。
主要的步骤：
1. 初始化LLM
2. 构建Graph
- 构建 state：state包含Schema和Reducer
- 构建Node
- 利用条件边创建循环
- 将Node连成Graph

In [5]:
from typing import TypedDict, List, Annotated
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
load_dotenv()

llm = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    temperature=0.5,
)

# state的schema是AnyMessage的List，也就是可以装入任意类型的Message
# Annnotated就是我们实现为state指定Reducer的工具。Annotated后面的方括号中有两个参数，
# 第一个参数是我们的state的schema，第二个参数是我们的Reducer函数。
# 这里我们使用add_messages函数作为Reducer，它的作用是将新的Message添加到state的messages列表中。
class ChatState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

# 构建Node的Function
# 用户输入获取节点
def get_user_input(state: ChatState) -> ChatState:
    """这个节点获取用户输入并添加到消息历史中"""
    user_input = input("用户: ")
    return {"messages": [HumanMessage(content=user_input)]}

# 机器人节点
def Chat_Bot(state: ChatState) -> ChatState:
    """这个节点调用LLM并将响应添加到消息历史中"""
    messages = state["messages"]
    response = llm.invoke(messages)
    print(f"\n机器人: {response.content}")
    return {"messages": [AIMessage(content=response.content)]}

# 利用条件边去构造循环
def should_continue(state: ChatState) -> str:
    """这个节点根据用户输入判断是否继续循环"""
    # 获取最后一条消息
    messages = state["messages"]
    last_message = messages[-1]
    if isinstance(last_message, HumanMessage) and last_message.content.lower() == "退出":
        return "END"
    return "Chat_Bot"

# 构建Graph
graph = StateGraph(ChatState)
# 添加节点
graph.add_node("Chat_Bot", Chat_Bot)
graph.add_node("get_user_input", get_user_input)
# 添加边
graph.add_edge(START, "get_user_input")
graph.add_conditional_edges(
    "get_user_input",
    should_continue,
    {
        "Chat_Bot": "Chat_Bot",
        "END": END,
    }
)
graph.add_edge("Chat_Bot", "get_user_input")
app = graph.compile()
# 可视化 Graph
# display(Image(app.get_graph().draw_mermaid_png()))

app.invoke({"messages": []})



机器人: 你好！😊 有什么我可以帮助你的吗？比如解答问题、提供建议，或者只是闲聊？欢迎告诉我你的需求！

机器人: 你好啦啦啦！😊 我是通义千问，你可以叫我Qwen。很高兴认识你，有什么问题或者想聊的话题吗？

机器人: 哈哈，我认识啦啦啦！😊 你朋友叫哈哈吗？听起来很有趣的名字！你们是不是经常一起开玩笑或者玩闹？如果需要，我也可以陪你一起聊聊哈哈，或者帮你分析一下这个名字的含义哦～你想聊什么呢？

机器人: 你叫啦啦啦！😊 之前你已经告诉我了，是不是觉得这个名字很有趣？如果你愿意，我们可以一起聊聊关于这个名字的故事或者想法～有什么想分享的吗？

机器人: 你的朋友叫哈哈！😊 你是不是觉得这个名字特别有意思？如果愿意的话，我们可以一起聊聊你们之间的趣事，或者你朋友的性格特点～想说点什么呢？


{'messages': [HumanMessage(content='你好啊', additional_kwargs={}, response_metadata={}, id='841405cf-3096-4aca-81e2-483b0a752a44'),
  AIMessage(content='你好！😊 有什么我可以帮助你的吗？比如解答问题、提供建议，或者只是闲聊？欢迎告诉我你的需求！', additional_kwargs={}, response_metadata={}, id='f23beade-c229-4632-8bdf-a0a0e6edd025', tool_calls=[], invalid_tool_calls=[]),
  HumanMessage(content='我叫啦啦啦。你叫什么啊', additional_kwargs={}, response_metadata={}, id='a69f3eec-d908-4d16-8738-7f92b86a58ea'),
  AIMessage(content='你好啦啦啦！😊 我是通义千问，你可以叫我Qwen。很高兴认识你，有什么问题或者想聊的话题吗？', additional_kwargs={}, response_metadata={}, id='4787ab06-4363-4199-aa34-eb57abcc7804', tool_calls=[], invalid_tool_calls=[]),
  HumanMessage(content='我有个朋友叫哈哈，你认识它吗', additional_kwargs={}, response_metadata={}, id='ecd62e98-9395-4931-b6e1-4fb721bfbb2e'),
  AIMessage(content='哈哈，我认识啦啦啦！😊 你朋友叫哈哈吗？听起来很有趣的名字！你们是不是经常一起开玩笑或者玩闹？如果需要，我也可以陪你一起聊聊哈哈，或者帮你分析一下这个名字的含义哦～你想聊什么呢？', additional_kwargs={}, response_metadata={}, id='8923136c-cd48-436b-9931-a3ade8e262fb', tool_calls=[], invalid_